# Day10_0 별첨: 고급 A/B 테스트 주제
# Advanced A/B Test Topics

---

## 📚 학습 목표

이 별첨 자료는 Day10 본문에서 다루지 않은 **고급 A/B 테스트 주제**를 다룹니다.

**다루는 내용:**
1. 비열등성 테스트 (Non-inferiority Test)
2. 연속형 변수 Welch's t-test 상세
3. 단측 vs 양측 검정 심화
4. MDE(Minimum Detectable Effect) 심화

**대상 독자:**
- 의료/제약/헬스케어 분야 데이터 분석가
- A/B 테스트 이론을 깊이 이해하고 싶은 분
- 연속형 지표(체류시간, 객단가 등) 분석이 많은 분

In [ ]:
# 필수 라이브러리
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.power import TTestIndPower, NormalIndPower
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 시드 고정
np.random.seed(42)

---

# Part 1: 비열등성 테스트 (Non-inferiority Test)

---

## 1.1 비열등성 테스트란?

### 일반적인 A/B 테스트 (우월성 검정)
- **목표**: B안이 A안보다 **더 낫다**는 것을 증명
- **가설**: H₀: μB ≤ μA vs H₁: μB > μA

### 비열등성 테스트 (Non-inferiority Test)
- **목표**: B안이 A안보다 **크게 나쁘지 않다**는 것을 증명
- **가설**: H₀: μA - μB ≥ Δ vs H₁: μA - μB < Δ (Δ = 허용 마진)

| 구분 | 우월성 검정 | 비열등성 검정 |
|------|------------|---------------|
| **목표** | B > A 증명 | B ≥ A - Δ 증명 |
| **귀무가설** | B ≤ A | B < A - Δ |
| **사용 상황** | 개선 효과 확인 | 비용 절감, 부작용 감소 시 |
| **주요 분야** | 마케팅, UX | 의료, 제약, 헬스케어 |

## 1.2 언제 비열등성 테스트를 사용하나?

### 실무 사례

**1. 제약/의료**
- 신약이 기존 약보다 **부작용은 적으면서** 효과는 비슷한지 확인
- 저렴한 제네릭 약이 오리지널 약과 효과가 비슷한지 확인

**2. 비용 절감 프로젝트**
- 저렴한 서버로 교체해도 응답 속도가 크게 느려지지 않는지
- 광고비를 줄여도 전환율이 크게 떨어지지 않는지

**3. UX 간소화**
- 회원가입 단계를 줄여도 가입률이 크게 떨어지지 않는지
- 추천 알고리즘을 단순화해도 클릭률이 크게 떨어지지 않는지

## 1.3 비열등성 마진 (Δ) 설정

비열등성 테스트의 핵심은 **허용 마진 Δ(델타)**를 설정하는 것입니다.

### 마진 설정 기준

1. **임상적/비즈니스적 의미**: 이 정도 차이는 실무적으로 무시해도 되는가?
2. **기존 효과의 일정 비율**: 보통 기존 효과의 10~20%
3. **규제 기준**: 의료 분야는 FDA 등의 가이드라인 참고

### 예시
- 기존 전환율 10% → 비열등성 마진 1%p (10%의 10%)
- 기존 체류시간 5분 → 비열등성 마진 30초 (10%)

In [ ]:
# 비열등성 테스트 시각화: 개념 이해

fig = go.Figure()

# 기준선 (A안 효과)
fig.add_shape(
    type="line", x0=0, x1=0, y0=0, y1=1,
    line=dict(color="black", width=2, dash="solid"),
)

# 비열등성 마진
fig.add_shape(
    type="line", x0=-1.5, x1=-1.5, y0=0, y1=1,
    line=dict(color="red", width=2, dash="dash"),
)

# 영역 표시
fig.add_vrect(x0=-5, x1=-1.5, fillcolor="red", opacity=0.2, line_width=0)
fig.add_vrect(x0=-1.5, x1=0, fillcolor="yellow", opacity=0.2, line_width=0)
fig.add_vrect(x0=0, x1=5, fillcolor="green", opacity=0.2, line_width=0)

# 시나리오 표시 (신뢰구간)
scenarios = [
    {"name": "시나리오 1: 비열등성 입증", "ci": (-1, 0.5), "y": 0.8, "color": "green"},
    {"name": "시나리오 2: 비열등성 입증 (경계)", "ci": (-1.4, 0.2), "y": 0.6, "color": "orange"},
    {"name": "시나리오 3: 판단 불가", "ci": (-2, 0.5), "y": 0.4, "color": "gray"},
    {"name": "시나리오 4: 열등", "ci": (-3, -1.8), "y": 0.2, "color": "red"},
]

for s in scenarios:
    fig.add_trace(go.Scatter(
        x=[s["ci"][0], s["ci"][1]],
        y=[s["y"], s["y"]],
        mode="lines+markers",
        name=s["name"],
        line=dict(color=s["color"], width=4),
        marker=dict(size=10)
    ))

fig.update_layout(
    title="비열등성 테스트: 신뢰구간 해석",
    xaxis_title="B - A (효과 차이)",
    yaxis_title="",
    yaxis=dict(showticklabels=False),
    xaxis=dict(range=[-4, 3]),
    annotations=[
        dict(x=0, y=1.05, text="A안 기준 (0)", showarrow=False),
        dict(x=-1.5, y=1.05, text="비열등성 마진 (-Δ)", showarrow=False, font=dict(color="red")),
        dict(x=-3, y=0.95, text="열등 영역", showarrow=False, font=dict(color="red")),
        dict(x=-0.75, y=0.95, text="비열등 영역", showarrow=False, font=dict(color="orange")),
        dict(x=2, y=0.95, text="우월 영역", showarrow=False, font=dict(color="green")),
    ],
    showlegend=True,
    height=400
)

fig.show()

### 신뢰구간 해석

| 시나리오 | 신뢰구간 | 결론 |
|----------|----------|------|
| 1 | 전체가 -Δ 오른쪽 | **비열등성 입증** |
| 2 | 하한이 -Δ 바로 오른쪽 | **비열등성 입증** (경계) |
| 3 | -Δ를 포함 | **판단 불가** (표본 크기 부족) |
| 4 | 전체가 -Δ 왼쪽 | **열등** (B안이 더 나쁨) |

In [ ]:
def non_inferiority_test_proportion(n1, x1, n2, x2, margin, alpha=0.05):
    """
    비율에 대한 비열등성 테스트
    
    Parameters:
    -----------
    n1, x1 : 대조군 (A안) 표본 크기, 성공 수
    n2, x2 : 실험군 (B안) 표본 크기, 성공 수
    margin : 비열등성 마진 (음수, 예: -0.01)
    alpha : 유의수준
    
    Returns:
    --------
    dict : 테스트 결과
    """
    # 비율 계산
    p1 = x1 / n1  # 대조군
    p2 = x2 / n2  # 실험군
    diff = p2 - p1  # B - A
    
    # 표준오차
    se = np.sqrt(p1 * (1 - p1) / n1 + p2 * (1 - p2) / n2)
    
    # 신뢰구간 (단측)
    z_alpha = stats.norm.ppf(1 - alpha)
    ci_lower = diff - z_alpha * se
    
    # 비열등성 판정
    # H0: p2 - p1 <= margin (열등)
    # H1: p2 - p1 > margin (비열등)
    z_stat = (diff - margin) / se
    p_value = 1 - stats.norm.cdf(z_stat)
    
    is_non_inferior = ci_lower > margin
    
    return {
        'p1_control': p1,
        'p2_treatment': p2,
        'difference': diff,
        'margin': margin,
        'ci_lower': ci_lower,
        'z_stat': z_stat,
        'p_value': p_value,
        'is_non_inferior': is_non_inferior,
        'conclusion': '비열등성 입증' if is_non_inferior else '비열등성 입증 실패'
    }


# 예시: 회원가입 프로세스 간소화
# A안 (기존): 5단계, B안 (신규): 3단계
# 질문: 단계를 줄여도 가입률이 크게 떨어지지 않는가?

n_control = 5000  # A안 노출
n_treatment = 5000  # B안 노출
conversions_control = 500  # A안 가입 (10%)
conversions_treatment = 480  # B안 가입 (9.6%)

# 비열등성 마진: 1%p (10%의 10%)
margin = -0.01

result = non_inferiority_test_proportion(
    n_control, conversions_control,
    n_treatment, conversions_treatment,
    margin=margin
)

print("="*50)
print("비열등성 테스트 결과: 회원가입 프로세스 간소화")
print("="*50)
print(f"A안 (기존 5단계) 전환율: {result['p1_control']:.2%}")
print(f"B안 (신규 3단계) 전환율: {result['p2_treatment']:.2%}")
print(f"차이 (B - A): {result['difference']:.2%}")
print(f"비열등성 마진: {result['margin']:.2%}")
print(f"신뢰구간 하한: {result['ci_lower']:.2%}")
print(f"p-value: {result['p_value']:.4f}")
print(f"\n결론: {result['conclusion']}")
print("="*50)

## 1.4 비열등성 테스트 샘플 사이즈 계산

비열등성 테스트는 일반 A/B 테스트보다 **더 많은 표본**이 필요합니다.

In [ ]:
def sample_size_non_inferiority(p1, margin, alpha=0.05, power=0.8):
    """
    비열등성 테스트를 위한 샘플 사이즈 계산 (비율)
    
    가정: p2 = p1 (실제로 동등할 때)
    """
    p2 = p1  # 비열등성 가정
    
    z_alpha = stats.norm.ppf(1 - alpha)
    z_beta = stats.norm.ppf(power)
    
    # 표준 공식
    numerator = (z_alpha + z_beta) ** 2 * (p1 * (1 - p1) + p2 * (1 - p2))
    denominator = margin ** 2
    
    n = numerator / denominator
    return int(np.ceil(n))


# 기존 전환율 10%, 비열등성 마진 1%p
p1 = 0.10
margin = 0.01  # 양수로 입력 (공식용)

n_ni = sample_size_non_inferiority(p1, margin)

# 비교: 일반 우월성 테스트 (MDE 1%p)
from statsmodels.stats.power import NormalIndPower
power_analysis = NormalIndPower()
effect_size = 0.01 / np.sqrt(p1 * (1 - p1))
n_superiority = power_analysis.solve_power(
    effect_size=effect_size, 
    alpha=0.05, 
    power=0.8, 
    alternative='two-sided'
)

print("샘플 사이즈 비교")
print("="*50)
print(f"기존 전환율: {p1:.0%}")
print(f"마진/MDE: 1%p")
print(f"\n비열등성 테스트: {n_ni:,}명 (그룹당)")
print(f"우월성 테스트:   {int(n_superiority):,}명 (그룹당)")
print(f"\n비열등성 테스트가 약 {n_ni / n_superiority:.1f}배 더 많은 표본 필요")

---

# Part 2: 연속형 변수 Welch's t-test 상세

---

## 2.1 연속형 지표 A/B 테스트

Day10 본문에서는 주로 **전환율(비율)**을 다뤘습니다. 실무에서는 연속형 지표도 많이 사용합니다:

| 지표 유형 | 예시 | 적합한 검정 |
|----------|------|------------|
| **비율 (이산형)** | 전환율, CTR, 가입률 | Z-test |
| **평균 (연속형)** | 체류시간, 객단가, 페이지뷰 | **t-test** |

### Welch's t-test vs Student's t-test

| 구분 | Student's t-test | Welch's t-test |
|------|------------------|----------------|
| **가정** | 등분산 가정 | 이분산 허용 |
| **적용** | 두 그룹 분산이 같을 때 | 분산이 다를 수 있을 때 |
| **권장** | 제한적 | **일반적으로 권장** |

> **실무 팁**: A/B 테스트에서는 **Welch's t-test**를 기본으로 사용합니다. 등분산 가정이 위반되어도 안전합니다.

In [ ]:
# 예시 데이터: 체류시간 A/B 테스트
# A안: 기존 레이아웃, B안: 신규 레이아웃

np.random.seed(42)

# A안: 평균 120초, 표준편차 30초
group_A = np.random.normal(loc=120, scale=30, size=500)
group_A = np.clip(group_A, 10, 300)  # 10~300초 범위

# B안: 평균 128초, 표준편차 35초 (분산이 다름)
group_B = np.random.normal(loc=128, scale=35, size=500)
group_B = np.clip(group_B, 10, 300)

# 기초 통계량
print("체류시간 기초 통계량")
print("="*50)
print(f"A안 - 평균: {np.mean(group_A):.1f}초, 표준편차: {np.std(group_A):.1f}초")
print(f"B안 - 평균: {np.mean(group_B):.1f}초, 표준편차: {np.std(group_B):.1f}초")
print(f"차이: {np.mean(group_B) - np.mean(group_A):.1f}초")

In [ ]:
# 분포 시각화
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=group_A, name='A안 (기존)',
    opacity=0.7, nbinsx=30,
    marker_color='blue'
))

fig.add_trace(go.Histogram(
    x=group_B, name='B안 (신규)',
    opacity=0.7, nbinsx=30,
    marker_color='orange'
))

fig.update_layout(
    title='체류시간 분포 비교',
    xaxis_title='체류시간 (초)',
    yaxis_title='빈도',
    barmode='overlay',
    height=400
)

fig.show()

In [ ]:
# Welch's t-test 수행
t_stat, p_value = stats.ttest_ind(group_B, group_A, equal_var=False)  # Welch's t-test

print("Welch's t-test 결과")
print("="*50)
print(f"t-통계량: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"\n결론: {'통계적으로 유의함 (p < 0.05)' if p_value < 0.05 else '통계적으로 유의하지 않음'}")

In [ ]:
# 효과 크기 (Cohen's d) 계산
def cohens_d(group1, group2):
    """독립 표본 Cohen's d 계산"""
    n1, n2 = len(group1), len(group2)
    var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    
    # Pooled standard deviation
    pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
    
    return (np.mean(group2) - np.mean(group1)) / pooled_std


d = cohens_d(group_A, group_B)

print(f"Cohen's d: {d:.3f}")
if abs(d) < 0.2:
    effect_size_label = "작은 효과 (small)"
elif abs(d) < 0.5:
    effect_size_label = "작은~중간 효과"
elif abs(d) < 0.8:
    effect_size_label = "중간 효과 (medium)"
else:
    effect_size_label = "큰 효과 (large)"
    
print(f"효과 크기 해석: {effect_size_label}")

In [ ]:
# 신뢰구간 계산 (평균 차이)
def mean_diff_confidence_interval(group1, group2, confidence=0.95):
    """
    두 그룹 평균 차이의 신뢰구간 계산 (Welch-Satterthwaite)
    """
    n1, n2 = len(group1), len(group2)
    mean1, mean2 = np.mean(group1), np.mean(group2)
    var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    
    # 표준오차
    se = np.sqrt(var1/n1 + var2/n2)
    
    # 자유도 (Welch-Satterthwaite)
    df = (var1/n1 + var2/n2)**2 / (
        (var1/n1)**2/(n1-1) + (var2/n2)**2/(n2-1)
    )
    
    # t 임계값
    t_crit = stats.t.ppf((1 + confidence) / 2, df)
    
    # 신뢰구간
    diff = mean2 - mean1
    ci_lower = diff - t_crit * se
    ci_upper = diff + t_crit * se
    
    return diff, ci_lower, ci_upper, se


diff, ci_lower, ci_upper, se = mean_diff_confidence_interval(group_A, group_B)

print("평균 차이 신뢰구간")
print("="*50)
print(f"평균 차이 (B - A): {diff:.2f}초")
print(f"95% 신뢰구간: [{ci_lower:.2f}, {ci_upper:.2f}]초")
print(f"표준오차: {se:.2f}초")

## 2.2 연속형 변수 샘플 사이즈 계산

비율과 달리, 연속형 변수의 샘플 사이즈는 **표준편차**가 필요합니다.

In [ ]:
# 연속형 변수 샘플 사이즈 계산
from statsmodels.stats.power import TTestIndPower

def sample_size_continuous(mean_diff, std, alpha=0.05, power=0.8):
    """
    연속형 변수 A/B 테스트 샘플 사이즈 계산
    
    Parameters:
    -----------
    mean_diff : 감지하고자 하는 평균 차이
    std : 표준편차 (두 그룹 동일하다고 가정)
    """
    effect_size = mean_diff / std  # Cohen's d
    
    power_analysis = TTestIndPower()
    n = power_analysis.solve_power(
        effect_size=effect_size,
        alpha=alpha,
        power=power,
        alternative='two-sided'
    )
    
    return int(np.ceil(n)), effect_size


# 예시: 객단가 A/B 테스트
# 현재 평균 객단가: 50,000원, 표준편차: 20,000원
# 목표: 5,000원 증가 감지 (10% 상승)

mean_diff = 5000  # 감지하고자 하는 차이
std = 20000  # 표준편차

n_per_group, effect_size = sample_size_continuous(mean_diff, std)

print("연속형 변수 샘플 사이즈 계산")
print("="*50)
print(f"현재 평균 객단가: ₩50,000")
print(f"표준편차: ₩{std:,}")
print(f"감지하고자 하는 차이: ₩{mean_diff:,}")
print(f"\nCohen's d (효과 크기): {effect_size:.2f}")
print(f"필요 샘플 크기: {n_per_group:,}명 (그룹당)")
print(f"총 필요 샘플: {n_per_group * 2:,}명")

In [ ]:
# 다양한 효과 크기에 따른 샘플 사이즈 비교
effect_sizes = [0.1, 0.2, 0.3, 0.5, 0.8]
sample_sizes = []

power_analysis = TTestIndPower()

for es in effect_sizes:
    n = power_analysis.solve_power(
        effect_size=es,
        alpha=0.05,
        power=0.8,
        alternative='two-sided'
    )
    sample_sizes.append(int(np.ceil(n)))

# 시각화
fig = go.Figure()

fig.add_trace(go.Bar(
    x=[f'd={es}' for es in effect_sizes],
    y=sample_sizes,
    text=sample_sizes,
    textposition='outside',
    marker_color=['#ff6b6b', '#ffa94d', '#ffd43b', '#69db7c', '#4dabf7']
))

fig.update_layout(
    title="효과 크기별 필요 샘플 사이즈 (그룹당)",
    xaxis_title="Cohen's d (효과 크기)",
    yaxis_title="필요 샘플 크기",
    yaxis_type="log",
    height=400
)

fig.show()

print("\n효과 크기 해석 기준:")
print("d=0.2: 작은 효과 (small)")
print("d=0.5: 중간 효과 (medium)")
print("d=0.8: 큰 효과 (large)")

---

# Part 3: 단측 vs 양측 검정 심화

---

## 3.1 단측 검정 vs 양측 검정

### 양측 검정 (Two-tailed test)
- **가설**: H₀: μA = μB vs H₁: μA ≠ μB
- **질문**: "두 그룹이 **다른가**?"
- **사용 상황**: 방향을 모를 때, 일반적인 경우

### 단측 검정 (One-tailed test)
- **가설**: H₀: μB ≤ μA vs H₁: μB > μA
- **질문**: "B가 A보다 **더 큰가**?"
- **사용 상황**: 방향이 명확할 때 (개선만 관심)

| 구분 | 양측 검정 | 단측 검정 |
|------|----------|----------|
| **방향성** | 양방향 | 단방향 |
| **유의수준 분배** | α/2 양쪽 | α 한쪽 |
| **검정력** | 낮음 | **높음** |
| **필요 표본** | 많음 | **적음** |

In [ ]:
# 단측 vs 양측 검정 시각화
from scipy.stats import norm

x = np.linspace(-4, 4, 1000)
y = norm.pdf(x)

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['양측 검정 (Two-tailed)', '단측 검정 (One-tailed, 우측)']
)

# 양측 검정
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='분포', line=dict(color='blue')), row=1, col=1)
# 기각역 (양쪽 2.5%)
x_left = x[x < -1.96]
x_right = x[x > 1.96]
fig.add_trace(go.Scatter(x=x_left, y=norm.pdf(x_left), fill='tozeroy', 
                         fillcolor='rgba(255,0,0,0.3)', name='기각역 (α/2=2.5%)', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=x_right, y=norm.pdf(x_right), fill='tozeroy', 
                         fillcolor='rgba(255,0,0,0.3)', showlegend=False, line=dict(color='red')), row=1, col=1)

# 단측 검정 (우측)
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='분포', line=dict(color='blue'), showlegend=False), row=1, col=2)
# 기각역 (오른쪽 5%)
x_right_one = x[x > 1.645]
fig.add_trace(go.Scatter(x=x_right_one, y=norm.pdf(x_right_one), fill='tozeroy', 
                         fillcolor='rgba(255,0,0,0.3)', name='기각역 (α=5%)', line=dict(color='red')), row=1, col=2)

fig.update_layout(height=350, title_text="단측 vs 양측 검정의 기각역")
fig.update_xaxes(title_text="z-score", row=1, col=1)
fig.update_xaxes(title_text="z-score", row=1, col=2)
fig.show()

print("\n임계값 비교 (α=0.05):")
print(f"양측 검정: |z| > {norm.ppf(0.975):.3f}")
print(f"단측 검정: z > {norm.ppf(0.95):.3f}")

In [ ]:
# 단측 vs 양측: 필요 샘플 사이즈 비교
power_analysis = TTestIndPower()

effect_size = 0.3  # 중간 정도의 효과

# 양측 검정
n_two_sided = power_analysis.solve_power(
    effect_size=effect_size,
    alpha=0.05,
    power=0.8,
    alternative='two-sided'
)

# 단측 검정
n_one_sided = power_analysis.solve_power(
    effect_size=effect_size,
    alpha=0.05,
    power=0.8,
    alternative='larger'  # B > A 검정
)

print("필요 샘플 사이즈 비교 (Cohen's d = 0.3)")
print("="*50)
print(f"양측 검정: {int(np.ceil(n_two_sided)):,}명 (그룹당)")
print(f"단측 검정: {int(np.ceil(n_one_sided)):,}명 (그룹당)")
print(f"\n단측 검정이 약 {n_two_sided/n_one_sided:.0%} 적은 표본 필요")

## 3.2 언제 단측 검정을 사용하나?

### 단측 검정 적합한 경우
1. **개선 방향이 명확**할 때
   - 전환율 **상승**만 관심
   - 로딩 시간 **감소**만 관심

2. **반대 방향은 의미 없을 때**
   - B안이 더 나쁘면 어차피 적용 안 할 것

3. **표본 수 제약**이 있을 때
   - 단측 검정이 더 적은 표본으로 검정력 확보

### 양측 검정 유지해야 하는 경우
1. **부정적 효과도 알아야** 할 때
   - B안이 얼마나 나쁜지도 정량화 필요

2. **규제 요건**이 있을 때
   - FDA 등 규제 기관은 양측 검정 요구

3. **논문/학술 목적**일 때
   - 학계에서는 양측 검정이 표준

> **실무 팁**: 대부분의 마케팅/UX A/B 테스트는 **양측 검정**을 사용합니다. 단측 검정은 사전에 명확한 근거가 있을 때만 사용하세요.

---

# Part 4: MDE(Minimum Detectable Effect) 심화

---

## 4.1 MDE란?

**MDE (Minimum Detectable Effect)**: 주어진 표본 크기로 감지할 수 있는 **최소 효과 크기**

### MDE 결정 요소
1. **표본 크기 (n)**: n↑ → MDE↓ (더 작은 효과 감지 가능)
2. **기준 비율/분산**: 기준값↑ → MDE 영향
3. **유의수준 (α)**: α↑ → MDE↓
4. **검정력 (1-β)**: Power↑ → MDE↑

### 공식 (비율)
$$MDE = (z_{\alpha/2} + z_{\beta}) \times \sqrt{\frac{2p(1-p)}{n}}$$

In [ ]:
def calculate_mde_proportion(n, baseline_rate, alpha=0.05, power=0.8):
    """
    비율에 대한 MDE 계산
    
    Parameters:
    -----------
    n : 그룹당 표본 크기
    baseline_rate : 기준 전환율
    alpha : 유의수준
    power : 검정력
    """
    z_alpha = stats.norm.ppf(1 - alpha/2)  # 양측
    z_beta = stats.norm.ppf(power)
    
    p = baseline_rate
    se = np.sqrt(2 * p * (1 - p) / n)
    
    mde = (z_alpha + z_beta) * se
    
    return mde


# 예시: 현재 전환율 5%, 다양한 표본 크기
baseline = 0.05
sample_sizes = [1000, 2000, 5000, 10000, 20000, 50000]

mdes = [calculate_mde_proportion(n, baseline) for n in sample_sizes]
mde_percentages = [mde / baseline * 100 for mde in mdes]  # 상대적 MDE (%)

# 결과 테이블
df_mde = pd.DataFrame({
    '그룹당 표본': sample_sizes,
    'MDE (절대값)': [f"{mde:.2%}p" for mde in mdes],
    'MDE (상대값)': [f"{pct:.1f}%" for pct in mde_percentages]
})

print("표본 크기별 MDE (기준 전환율: 5%)")
print("="*50)
print(df_mde.to_string(index=False))

In [ ]:
# MDE 시각화
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['절대적 MDE', '상대적 MDE (% 개선)']
)

# 절대적 MDE
fig.add_trace(go.Scatter(
    x=sample_sizes,
    y=[mde * 100 for mde in mdes],  # %p로 변환
    mode='lines+markers',
    name='MDE',
    line=dict(color='blue', width=2)
), row=1, col=1)

# 상대적 MDE
fig.add_trace(go.Scatter(
    x=sample_sizes,
    y=mde_percentages,
    mode='lines+markers',
    name='상대적 MDE',
    line=dict(color='orange', width=2)
), row=1, col=2)

fig.update_xaxes(title_text="그룹당 표본 크기", type="log", row=1, col=1)
fig.update_xaxes(title_text="그룹당 표본 크기", type="log", row=1, col=2)
fig.update_yaxes(title_text="MDE (%p)", row=1, col=1)
fig.update_yaxes(title_text="MDE (% 개선)", row=1, col=2)

fig.update_layout(
    height=400,
    title_text="표본 크기 vs MDE (기준 전환율: 5%)",
    showlegend=False
)

fig.show()

## 4.2 MDE 실무 가이드라인

### 적절한 MDE 기준

| 상황 | 권장 MDE (상대값) | 근거 |
|------|------------------|------|
| **대규모 변경** | 10~20% | 큰 효과 기대 |
| **일반적인 개선** | 5~10% | 현실적인 개선 수준 |
| **미세 최적화** | 2~5% | 많은 표본 필요 |
| **매우 민감한 지표** | <2% | 대규모 트래픽 필요 |

### MDE 설정 실무 팁

1. **비즈니스 임팩트 기준**
   - 이 정도 개선이면 의미 있는가?
   - ROI가 나오는가?

2. **과거 실험 참고**
   - 비슷한 실험에서 어느 정도 효과가 있었는가?
   - 현실적으로 기대할 수 있는 수준은?

3. **표본 제약 고려**
   - 하루 트래픽이 얼마인가?
   - 실험 기간 제약은?

In [ ]:
# MDE 인터랙티브 계산기
def mde_calculator(daily_traffic, test_duration_days, baseline_rate, 
                   traffic_allocation=0.5, alpha=0.05, power=0.8):
    """
    실무용 MDE 계산기
    
    Parameters:
    -----------
    daily_traffic : 일일 트래픽
    test_duration_days : 실험 기간 (일)
    baseline_rate : 기준 전환율
    traffic_allocation : 실험에 할당하는 트래픽 비율
    """
    total_traffic = daily_traffic * test_duration_days * traffic_allocation
    n_per_group = total_traffic / 2  # A/B 균등 분배
    
    mde = calculate_mde_proportion(n_per_group, baseline_rate, alpha, power)
    relative_mde = mde / baseline_rate * 100
    
    return {
        'total_traffic': int(total_traffic),
        'n_per_group': int(n_per_group),
        'mde_absolute': mde,
        'mde_relative': relative_mde,
        'new_rate_upper': baseline_rate + mde,
        'new_rate_lower': baseline_rate - mde
    }


# 예시: 이커머스 사이트
result = mde_calculator(
    daily_traffic=10000,  # 일일 방문자 1만명
    test_duration_days=14,  # 2주 실험
    baseline_rate=0.03,  # 현재 전환율 3%
    traffic_allocation=1.0  # 전체 트래픽 사용
)

print("MDE 계산 결과")
print("="*50)
print(f"일일 트래픽: 10,000명")
print(f"실험 기간: 14일")
print(f"총 실험 참여자: {result['total_traffic']:,}명")
print(f"그룹당 표본: {result['n_per_group']:,}명")
print(f"\n기준 전환율: 3.00%")
print(f"MDE (절대값): {result['mde_absolute']:.2%}p")
print(f"MDE (상대값): {result['mde_relative']:.1f}%")
print(f"\n감지 가능 범위: {result['new_rate_lower']:.2%} ~ {result['new_rate_upper']:.2%}")

---

## 📊 학습 정리

---

### 핵심 개념 요약

| 주제 | 핵심 내용 | 사용 상황 |
|------|----------|----------|
| **비열등성 테스트** | B ≥ A - Δ 증명 | 비용 절감, 부작용 감소 확인 시 |
| **Welch's t-test** | 이분산 허용하는 t-검정 | 연속형 지표 (체류시간, 객단가) |
| **단측 검정** | 한 방향만 검정 | 개선 방향이 명확할 때 |
| **MDE** | 감지 가능한 최소 효과 | 실험 설계 시 표본 크기 결정 |

### 실무 적용 가이드

1. **일반적인 A/B 테스트**: Z-test (비율), 양측 검정
2. **연속형 지표**: Welch's t-test
3. **비용 절감 검증**: 비열등성 테스트 + 적절한 마진 설정
4. **표본 제약 시**: 단측 검정 검토 (근거 필요)

### 주의사항

- 비열등성 마진은 **사전에** 정의 (p-hacking 방지)
- 단측 검정은 **명확한 근거** 있을 때만 사용
- MDE는 **비즈니스 의미**와 **통계적 제약** 모두 고려

---

## 📚 참고 자료

### 추가 학습 자료
- [Evan Miller's A/B Testing Calculator](https://www.evanmiller.org/ab-testing/)
- [Optimizely Sample Size Calculator](https://www.optimizely.com/sample-size-calculator/)
- [FDA Guidance on Non-inferiority Trials](https://www.fda.gov/regulatory-information/search-fda-guidance-documents/non-inferiority-clinical-trials)

### Python 라이브러리
- `scipy.stats.ttest_ind()` - t-test
- `statsmodels.stats.power` - 검정력 분석
- `statsmodels.stats.proportion` - 비율 검정

---

**작성자**: Dante Labs  
**최종 수정**: 2026-01-12  
**버전**: 1.0